In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

CNN will be used due to its ability to recognise patterns within images.

Convolutional layers are the core building blocks that will examen the image. Each layer from 1-3 will become more complex and use a larger number of filters to detect wider variety of patterns and high level features.

Pooling layers simpify the information from the convolutional layers.

Flatten layer prepares the data for the final classification. It takes the 3D output from the previous layers and flattens it into a single, 1D vector.

Dense layers are the final decision making layers. "Dense(128, ...), this Dense layer has 128 neurons - each neuron is connected to all neurons from the previous flatten layer. These neurons learn to combine the features that have been extracted from the convolutional layers to make sense of the data.

The dropout layer aims to prevent overfitting. If the dropout rate is 0.5, during training it will randomly disable 50% of the neurons in the previous layer to force the network to learn more robust features. It also prevents it from relying on a small number of neurons to make a prediction.

In [3]:
# Function to build the CNN
def build_cnn(input_shape, num_classes):
    model = Sequential([
        # First Convolutional layer so small number of filters
        Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
        MaxPooling2D((2, 2)),

        # Second Convolutional layer - double filters
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),

        # Third Convolutional layer - double filters again
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax")
    ])

    return model

In [4]:
# Main execution
if __name__ == "__main__":


    print("Loading preprocessed data...")
    try:
        # Normalise whilst loading in
        x_train = np.load("../data/processed/x_train.npy") / 255.0
        y_train = np.load("../data/processed/y_train.npy")
        x_val = np.load("../data/processed/x_val.npy") / 255.0
        y_val = np.load("../data/processed/y_val.npy")
        x_test = np.load("../data/processed/x_test.npy") / 255.0
        y_test = np.load("../data/processed/y_test.npy")
        print("Data loaded successfully.")
    except FileNotFoundError:
        print("Error: One or more .npy file(s) could not be found.")

    # Extract neccessities for building model
    img_size = x_train.shape[1]
    num_classes = len(np.unique(np.concatenate([y_train, y_val, y_test])))
    input_shape = (img_size, img_size, 3)

    # Build model
    model = build_cnn(input_shape, num_classes)
    #model.summary()

    # Compile the model
    model.compile(
        optimizer = Adam(learning_rate = 0.001),
        loss = "sparse_categorical_crossentropy",
        metrics = ["accuracy"]
    )

    # Define callbacks
    early_stopping = EarlyStopping(
        monitor = "val_loss",
        patience = 10,
        restore_best_weights=True
    )

    # Train the model
    print("\nStarting model training...")
    model.fit(
        x_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )
    print("\nModel training completed.")

    # Evaluate model
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test loss: {loss:.4f}")
    print(f"Test accuracy: {accuracy:.4f}")
    
    # Save model
    model.save('../model/skin_lesion_classifier_model.keras')
    print("\nModel saved successfully.")


Loading preprocessed data...
Data loaded successfully.


c:\Users\Chazf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Starting model training...
Epoch 1/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 23s 97ms/step - accuracy: 0.6632 - loss: 1.0619 - val_accuracy: 0.6698 - val_loss: 0.9480
Epoch 2/50
 71/220 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.6793 - loss: 0.9585

KeyboardInterrupt: 